In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# PyTorch tutorial
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 3)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # x = F.softmax(self.fc3(x), dim=1)
        return x

In [ ]:
# Rosebrock
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(3, 32, 3, 1, 1)
        self.fc = nn.Linear(32 * 32 * 32, 3)
        
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = x.view(-1, 32 * 32 * 32)
        x = F.softmax(self.fc(x), dim=1)
        return x

In [ ]:
class AnimalsDataset(Dataset):
    
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset[0])
    
    def __getitem__(self, idx):
        image= self.dataset[0][idx]
        target = self.dataset[1][idx]
        
        if self.transform:
            image = self.transform(image)
            
        return [image, target]

In [ ]:
def read_image(root_dir, test_size=0.25, validation_size=0.1, batch_size=32, transform=None):
    # Read images and labels
    data, labels = [], []
    classes = [c for c in os.listdir(root_dir) if not c.startswith('.')]
    paths = [os.path.join(root_dir, c) for c in classes]
    for path in paths:
        files = os.listdir(path)
        filenames = [os.path.join(path, file) for file in files]
        data.extend([Image.open(f).convert('RGB') for f in filenames])
        labels.extend([classes.index(f.split('_')[0]) for f in files])
    
    # Split data into trainset and testset
    X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=test_size, random_state=42)
    trainset = (X_train, Y_train)
    testset = (X_test, Y_test)

    # Transform RGB image to torch.Tensor
    if not transform:
        transform = transforms.Compose([
            transforms.Resize((32, 32)), # resize all the image to 32x32x3
            transforms.ToTensor(),       # rescale images from [0, 255] to [0, 1.0], (H x W x C) to (C x H x W)
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # rescale [0, 1.0] to [-1.0, 1.0]
        ])
    trainset, testset = AnimalsDataset(trainset, transform), AnimalsDataset(testset, transform)

    # Create DataLoader for trainset and testset
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=4)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return (trainloader, testloader, classes)

In [ ]:
def CNN(dataset, epochs=50, lr=1e2, momentum=8.5e-1, dest='../../trained_model'):
    trainloader, testloader, classes = dataset
    batch_size = trainloader.batch_size
    # Instantiate CNN, pick loss function and optimizer
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum) # lr: 1e-1, 1e-2, 1e-3, 1e-4; momentum: 0.9-0.99

    for epoch in range(epochs):
        running_loss = 0.0
        for train_batch in trainloader:
            images, targets = train_batch
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() / batch_size
        print('[epoch {:02d}/{:2d}] loss: {:5.3f}'.format(epoch + 1, epochs, running_loss))
    print('Finished training.')
    if not os.path.exists(dest):
        os.mkdir(dest)
    model_filename = '{:s}/animals_{:.1e}_{:.1e}_{:d}.pth'.format(dest, lr, momentum, epochs)
    torch.save(net.state_dict(), model_filename)
    print('Saving trained model to {:s}'.format(model_filename))

    # Calculate the accuracy and generate classification report
    number_of_classes = len(classes)
    with torch.no_grad():
        class_correct = [0. for i in range(number_of_classes)]
        class_total = [0. for i in range(number_of_classes)]
        acc = [0. for i in range(number_of_classes)]
        predicts_total, targets_total = [], []
        for test_batch in testloader:
            images, targets = test_batch
            outputs = net(images)
            _, predicts = torch.max(outputs, 1)
            predicts_total.extend(predicts.numpy())
            targets_total.extend(targets.numpy())
            c = (predicts == targets).squeeze()
            for i, target in enumerate(targets):
                class_correct[target] += c[i].item()
                class_total[target] += 1
        report = classification_report(targets_total, predicts_total, target_names=classes)

        for i in range(3):
            acc[i] = 100 * class_correct[i] / class_total[i]
        
    return (report, acc)

In [ ]:
root_dir = '/Users/libao/Documents/data/animals/'
dataset = read_image(root_dir)
trainloader, testloader, classes = dataset

In [ ]:
epochs = 50
learning_rates = [1e-1, 1e-2,1e-3, 1e-4]
momentums = torch.arange(0.8, 1.00, 0.05)

batch_size = 32
for lr in learning_rates:
    for momentum in momentums:
        print('learning rate: {:e}, momentumn: {:e}'.format(lr, momentum))
        report, accuracies = CNN(dataset=dataset, epochs=epochs, lr=lr, momentum=momentum)

        # for i, acc in enumerate(accuracies):
        #     print('Accuracy of {:5s} : {:.2f} %'.format(classes[i], acc))
        print('Classification report:\n', report)

In [ ]:
def imshow(image):
    image = image / 2 + 0.5
    
    # Method 1
    image_pil = transforms.ToPILImage()(image)
    plt.imshow(image_pil)
    
    # # Method 2
    # image_np = image.numpy()
    # plt.imshow(np.transpose(image_np, (1, 2, 0)))
    
    plt.show()

# transform = transforms.Compose([
#     transforms.Resize((32,32)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])
# image = Image.open('{:s}/cats/cats_00001.jpg'.format(root_dir))
# image = transform(image)
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(labels.shape)
print(labels)
print(images[0].shape)    
imshow(torchvision.utils.make_grid(images))
print(images[0].dtype)
print(' '.join('{:5s}'.format(classes[labels[j]]) for j in range(4)))

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('{:5s}'.format(classes[labels[j]]) for j in range(4)))

net = Net().float()
PATH = './animals.pth'
net.load_state_dict(torch.load(PATH))
outputs = net(images.float())
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('{:5s}'.format(classes[predicted[j]]) for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: {:f} %'.format(100 * correct / total))

In [ ]:
# alexnet = torchvision.models.alexnet(pretrained=True)
# vgg16 = torchvision.models.vgg16(pretrained=True)
# resnet101 = torchvision.models.resnet101(pretrained=True)